<a href="https://colab.research.google.com/github/jonathancagua/NLP/blob/main/EX/Desafio_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Datos
El objecto es utilizar datos disponibles del challenge ConvAI2 (Conversational Intelligence Challenge 2) de conversaciones en inglés. Se construirá un BOT para responder a preguntas del usuario (QA).\
[LINK](http://convai.io/data/)

In [1]:
import pandas as pd
import requests
import json

# URL del dataset ConvAI2
url = "http://convai.io/data/summer_wild_evaluation_dialogs.json"

try:
    # Descargar el dataset
    response = requests.get(url)
    response.raise_for_status()  # Lanza excepción si hubo error HTTP
    convai2_json = response.json()
except requests.exceptions.RequestException as e:
    raise SystemExit(f"Error al descargar el dataset: {e}")
except json.JSONDecodeError:
    raise SystemExit("Error al decodificar el JSON recibido.")

# Extraer pares pregunta-respuesta
questions, answers = [], []

for dialog in convai2_json:
    utterances = dialog.get('dialog', [])
    for i in range(len(utterances) - 1):
        q = utterances[i].get('text', '').strip()
        a = utterances[i + 1].get('text', '').strip()
        if q and a:  # Ignorar si alguno está vacío
            questions.append(q)
            answers.append(a)

# Crear DataFrame con los primeros 15,000 pares
df = pd.DataFrame({'question': questions, 'answer': answers})
df = df.iloc[:15000].copy()

# Vista previa
df.head()


,question,answer
0,I love iphone! i just bought new iphone!,"Thats good for you, i'm not very into new tech"
1,"Thats good for you, i'm not very into new tech",I am a college student and i am a college student
2,I am a college student and i am a college student,I am go to gym and live on donations
3,I am go to gym and live on donations,I am a vegan and i am in the midwest
4,I am a vegan and i am in the midwest,So vegan... i have dogs maybe i should told th...


In [2]:
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt_tab')
nltk.download('punkt')  # Tokenizador de palabras y frases


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import re
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize

# Diccionario ampliado de contracciones
CONTRACTIONS = {
    "i'm": "i am", "you're": "you are", "he's": "he is", "she's": "she is",
    "it's": "it is", "we're": "we are", "they're": "they are", "can't": "cannot",
    "won't": "will not", "don't": "do not", "didn't": "did not", "i've": "i have",
    "i'll": "i will", "you'll": "you will", "she'd": "she would", "should've": "should have",
    "there's": "there is", "we'd": "we would", "they'll": "they will", "wasn't": "was not",
    "isn't": "is not", "aren't": "are not", "couldn't": "could not", "wouldn't": "would not",
    "hasn't": "has not", "hadn't": "had not", "we'll": "we will", "they'd": "they would",
    "who's": "who is", "what's": "what is", "let's": "let us", "you've": "you have"
}

# Expande contracciones en el texto
def expand_contractions(text):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in CONTRACTIONS) + r')\b')
    return pattern.sub(lambda x: CONTRACTIONS[x.group()], text)

# Función de limpieza completa
def clean_text(text):
    text = text.lower()  # Minusculizar
    text = BeautifulSoup(text, "lxml").get_text()  # Eliminar HTML
    text = re.sub(r'http\S+', '', text)  # Eliminar URLs
    text = expand_contractions(text)  # Expandir contracciones
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuación
    text = re.sub(r'\d+', '', text)  # Eliminar números
    text = re.sub(r'\s+', ' ', text).strip()  # Normalizar espacios
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)  # Limitar letras repetidas (coooool → coool)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remover caracteres no ASCII
    text = re.sub(r'[\u2600-\u26FF\u263a-\U0001f645]', ' ', text)  # Remover emojis y símbolos Unicode
    tokens = word_tokenize(text)  # Tokenizar
    return ' '.join(tokens)

# Aplicar limpieza al DataFrame
df['question_clean'] = df['question'].astype(str).apply(clean_text)
df['answer_clean'] = df['answer'].astype(str).apply(clean_text)

# Agregar tokens de inicio y fin de secuencia
df['answer_clean'] = df['answer_clean'].apply(lambda x: '<sos> ' + x.strip() + ' <eos>')

# Filtrado y limpieza final
df = df.dropna(subset=['question_clean', 'answer_clean'])
df = df[(df['question_clean'].str.strip() != '') & (df['answer_clean'].str.strip() != '')]
df = df[df['question_clean'] != df['answer_clean']]
df = df[df['answer_clean'].str.split().apply(len) > 3]
df = df.drop_duplicates(subset=['question_clean', 'answer_clean'])

# Vista previa
print(f"Total de pares tras limpieza: {len(df)}")
display(df[['question_clean', 'answer_clean']].head())


Total de pares tras limpieza: 12647


,question_clean,answer_clean
0,i love iphone i just bought new iphone,<sos> thats good for you i am not very into ne...
1,thats good for you i am not very into new tech,<sos> i am a college student and i am a colleg...
2,i am a college student and i am a college student,<sos> i am go to gym and live on donations <eos>
3,i am go to gym and live on donations,<sos> i am a vegan and i am in the midwest <eos>
4,i am a vegan and i am in the midwest,<sos> so vegan i have dogs maybe i should told...


In [4]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parámetros
MAX_VOCAB_SIZE = 6000
MAX_SEQ_LENGTH = 25

# Tokenizadores
tokenizer_inputs = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='', lower=True, oov_token='<unk>')
tokenizer_inputs.fit_on_texts(df['question_clean'])
input_sequences = tokenizer_inputs.texts_to_sequences(df['question_clean'])

tokenizer_outputs = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='', lower=True, oov_token='<unk>')
tokenizer_outputs.fit_on_texts(df['answer_clean'])
output_sequences = tokenizer_outputs.texts_to_sequences(df['answer_clean'])

# Diccionarios de vocabulario
word2idx_inputs = tokenizer_inputs.word_index
word2idx_outputs = tokenizer_outputs.word_index

# Asegurarse de respetar el vocabulario limitado
num_words_output = min(MAX_VOCAB_SIZE, len(word2idx_outputs) + 1)

# Longitudes máximas de secuencia
max_input_len = min(MAX_SEQ_LENGTH, max(len(seq) for seq in input_sequences))
max_output_len = min(MAX_SEQ_LENGTH, max(len(seq) for seq in output_sequences))

# Padding (relleno con ceros al final)
encoder_input_sequences = pad_sequences(input_sequences, maxlen=max_input_len, padding='post')
decoder_input_sequences = pad_sequences(output_sequences, maxlen=max_output_len, padding='post')

# Targets del decoder (one-hot desplazados)
decoder_targets = np.zeros((len(decoder_input_sequences), max_output_len, num_words_output), dtype='float32')

for i, seq in enumerate(decoder_input_sequences):
    for t in range(1, len(seq)):
        word_idx = seq[t]
        if word_idx < num_words_output:
            decoder_targets[i, t - 1, word_idx] = 1.0

# Prints informativos
print(f"Total de pares pregunta-respuesta: {len(df)}")
print(f"Vocabulario input: {min(len(word2idx_inputs), MAX_VOCAB_SIZE)}")
print(f"Vocabulario output: {num_words_output}")
print(f"Longitud máxima input: {max_input_len}")
print(f"Longitud máxima output: {max_output_len}")
print(f"Shape de decoder_targets: {decoder_targets.shape}")

# Ejemplo ilustrativo
idx = 0
print("\n🔎 Ejemplo:")
print(f"Pregunta original: {df['question_clean'].iloc[idx]}")
print(f"Secuencia tokenizada: {input_sequences[idx]}")
print(f"Secuencia padded: {encoder_input_sequences[idx]}")
print(f"Respuesta tokenizada: {output_sequences[idx]}")
print(f"Respuesta padded: {decoder_input_sequences[idx]}")


Total de pares pregunta-respuesta: 12647
Vocabulario input: 4042
Vocabulario output: 4167
Longitud máxima input: 25
Longitud máxima output: 25
Shape de decoder_targets: (12647, 25, 4167)

🔎 Ejemplo:
Pregunta original: i love iphone i just bought new iphone
Secuencia tokenizada: [2, 21, 854, 2, 39, 739, 145, 854]
Secuencia padded: [  2  21 854   2  39 739 145 854   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0]
Respuesta tokenizada: [2, 54, 28, 27, 5, 4, 6, 10, 53, 231, 154, 2231, 3]
Respuesta padded: [   2   54   28   27    5    4    6   10   53  231  154 2231    3    0
    0    0    0    0    0    0    0    0    0    0    0]


In [5]:
pip install pandas requests numpy==1.26.4 gensim==4.3.3 nltk tensorflow==2.16.1


In [6]:
import numpy as np

import gensim.downloader as api
import gensim
from tqdm import tqdm

# Parámetros
EMBEDDING_MODEL = 'fasttext-wiki-news-subwords-300'
EMBEDDING_DIM = 300

def load_fasttext_model():
    print("Cargando modelo FastText...")
    try:
        return api.load(EMBEDDING_MODEL)
    except Exception as e:
        print(f"Error al cargar FastText: {e}")
        return None

def build_embedding_matrix(tokenizer, embedding_model, vocab_size, embedding_dim):
    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    found = 0
    missing_words = []

    print("Generando matriz de embeddings...")
    for word, i in tqdm(word_index.items(), total=len(word_index), desc="Procesando palabras"):
        if i >= vocab_size:
            continue
        if word in embedding_model:
            embedding_matrix[i] = embedding_model[word]
            found += 1
        else:
            missing_words.append(word)

    print(f"\n Palabras encontradas: {found}/{len(word_index)}")
    print(f"Palabras faltantes: {len(missing_words)} (ej: {missing_words[:10]})")
    print(f"Dimensión de la matriz: {embedding_matrix.shape}")

    return embedding_matrix

# Ejecución
fasttext_model = load_fasttext_model()
embedding_matrix = build_embedding_matrix(tokenizer_inputs, fasttext_model, MAX_VOCAB_SIZE, EMBEDDING_DIM)


Cargando modelo FastText...
[==================================================] 100.0% 958.5/958.4MB downloaded
Generando matriz de embeddings...


Procesando palabras: 100%|██████████| 4042/4042 [00:00<00:00, 106745.86it/s]


 Palabras encontradas: 3731/4042
Palabras faltantes: 311 (ej: ['<unk>', 'convai', 'whazzup', 'buongiorno', '_', 'poyou', 'zitah', 'orhun', 'wontice', 'hesnt'])
Dimensión de la matriz: (6000, 300)


### modelo Seq2Seq con atención Luong (dot-product)

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dot, Activation, Concatenate
from tensorflow.keras.optimizers import Adam

# Parámetros
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(max_input_len,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=MAX_VOCAB_SIZE,
                              output_dim=EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=max_input_len,
                              trainable=False,
                              name='encoder_embedding')(encoder_inputs)

encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_output_len,), name='decoder_inputs')
decoder_embedding = Embedding(input_dim=num_words_output,
                              output_dim=latent_dim,
                              name='decoder_embedding')(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.2, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Atención (Luong - dot product)
attention_scores = Dot(axes=[2, 2], name='attention_scores')([decoder_outputs, encoder_outputs])
attention_weights = Activation('softmax', name='attention_weights')(attention_scores)
context_vector = Dot(axes=[2, 1], name='context_vector')([attention_weights, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1, name='decoder_combined_context')([context_vector, decoder_outputs])

# Densas finales
dense_intermediate = Dense(256, activation='tanh', name='dense_intermediate')(decoder_combined_context)
decoder_outputs_final = Dense(num_words_output, activation='softmax', name='decoder_outputs')(dense_intermediate)

# Modelo final
model = Model([encoder_inputs, decoder_inputs], decoder_outputs_final)

# Compilación
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Resumen
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, 25)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, 25)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, 25, 300)   │  1,800,000 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, 25, 256)   │  1,066,752 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 25, 256), │    570,368 │ encoder_embeddin… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, 25, 256), │    525,312 │ decoder_embeddin… │
│                     │ (None, 256),      │            │ encoder_lstm[0][… │
│                     │ (None, 256)]      │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_scores    │ (None, 25, 25)    │          0 │ decoder_lstm[0][… │
│ (Dot)               │                   │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_weights   │ (None, 25, 25)    │          0 │ attention_scores… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ context_vector      │ (None, 25, 256)   │          0 │ attention_weight… │
│ (Dot)               │                   │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_combined_c… │ (None, 25, 512)   │          0 │ context_vector[0… │
│ (Concatenate)       │                   │            │ decoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_intermediate  │ (None, 25, 256)   │    131,328 │ decoder_combined… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_outputs     │ (None, 25, 4167)  │  1,070,919 │ dense_intermedia… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,164,679 (19.70 MB)

 Trainable params: 3,364,679 (12.84 MB)

 Non-trainable params: 1,800,000 (6.87 MB)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Parámetros de entrenamiento
BATCH_SIZE = 64
EPOCHS = 30
VALIDATION_SPLIT = 0.2
MODEL_PATH = 'best_model.keras'

# Callback: detener si no mejora val_loss en N épocas
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# Callback: guardar el mejor modelo basado en val_loss
checkpoint = ModelCheckpoint(
    filepath=MODEL_PATH,
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Entrenamiento
history = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=VALIDATION_SPLIT,
    callbacks=[early_stop, checkpoint],
    verbose=1
)


Epoch 1/30
 90/159 ━━━━━━━━━━━━━━━━━━━━ 2:02 2s/step - accuracy: 0.6214 - loss: 3.7324

### Inferencia

In [ ]:
# --- Encoder: devuelve encoder_outputs y los estados finales ---
encoder_model = Model(
    encoder_inputs,
    [encoder_outputs, state_h, state_c]
)

# --- Decoder: inputs paso a paso ---
decoder_input_token = Input(shape=(1,), name='decoder_input_token')
decoder_input_h = Input(shape=(latent_dim,), name='decoder_state_h')
decoder_input_c = Input(shape=(latent_dim,), name='decoder_state_c')
encoder_outputs_inf = Input(shape=(max_input_len, latent_dim), name='encoder_outputs_inf')

# Reusar la capa de embeddings
decoder_embedding_inf = decoder_embedding_layer(decoder_input_token)

# LSTM para un solo paso (step-by-step decoding)
decoder_lstm_outputs, decoder_h, decoder_c = decoder_lstm(
    decoder_embedding_inf,
    initial_state=[decoder_input_h, decoder_input_c]
)

# Atención Luong: dot-product
attention_scores = Dot(axes=[2, 2])([decoder_lstm_outputs, encoder_outputs_inf])
attention_weights = Activation('softmax')(attention_scores)
context_vector = Dot(axes=[2, 1])([attention_weights, encoder_outputs_inf])

# Concatenar contexto y salida del decoder
decoder_context_concat = Concatenate(axis=-1)([context_vector, decoder_lstm_outputs])

# Salida densa
decoder_tanh = Dense(256, activation='tanh')(decoder_context_concat)
decoder_output_probs = decoder_dense(decoder_tanh)

# Modelo final del decoder (para un paso)
decoder_model = Model(
    [decoder_input_token, decoder_input_h, decoder_input_c, encoder_outputs_inf],
    [decoder_output_probs, decoder_h, decoder_c]
)
